In [3]:
import pandas as pd
from os.path import basename
import re
import sqlparse
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as offline
import networkx as nx

Read sps calls from DB dependencies objects extract

In [33]:
deps_df = pd.read_csv("./data/sps_calls.csv")

deps_df.tail()

,source,target
685,spAPT_GuideExcursionDelete,spSystem_QInsert
686,spGuide_ExcursionListPList,spGuide_PListGetAll
687,spResource_ResourceGet,spUsage_usrResourceCreate
688,spGuide_usrCompletedTourGet,spCol_GetContentAccessTillDate
689,sp_usrForTeamListInactive,sp_GDescendantTeamusrsForLeader


Form nodes and edges then create directed graph

In [34]:
all_edges = []
for idx, row in deps_df.iterrows():
    all_edges.append((row["source"], row["target"]))

all_nodes = list(deps_df["source"].unique())
all_nodes.extend(list(deps_df["target"].unique()))
all_nodes = list(set(all_nodes))

G = nx.DiGraph()
G.add_nodes_from(all_nodes)
G.add_edges_from(all_edges)

In [42]:
pos = nx.spring_layout(G)
# pos = nx.kamada_kawai_layout(G)

In [43]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='Earth',
        reversescale=True,
        color=[],
        size=15,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

Add arrows to highlight what SP calls another SP

In [44]:
def add_arrows(edges, fig, pos):
    for edge in edges:
        x = pos[edge[1]][0]  # head
        y = pos[edge[1]][1]  # head
        ax = pos[edge[0]][0]
        ay = pos[edge[0]][1]
        fig.add_annotation(
            x=x,  # head
            y=y,
            ax=ax,  # tail
            ay=ay,
            showarrow=True,
            arrowcolor="grey",
            text="",
            xref='x',
            yref='y',
            axref='x',
            ayref='y',
            arrowwidth=1,
            arrowhead=2,
            arrowsize=2)

In [45]:
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(
        adjacencies[0] + ' has # of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [46]:
data = [edge_trace, node_trace]

layout = go.Layout(
    title='SPs Network Graph',
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False,
               zeroline=False,
               showticklabels=False),
    yaxis=dict(showgrid=False,
               zeroline=False,
               showticklabels=False))

fig = go.Figure(data=data, layout=layout)

add_arrows(G.edges(), fig, pos)

fig.show()

In [47]:
fig.write_html("./reports/sp-network-graph.html")